In [2]:
import numpy as np
import time
import matplotlib.pyplot as plt
import cv2
import depthai as dai
import os

In [56]:
FDIR='./data'
def getPipeLine(streamName):
    pipeline = dai.Pipeline()

    # Define sources and outputs
    camRgb = pipeline.create(dai.node.ColorCamera)
    monoLeft = pipeline.create(dai.node.MonoCamera)
    monoRight = pipeline.create(dai.node.MonoCamera)
    ve1 = pipeline.create(dai.node.VideoEncoder)
    ve2 = pipeline.create(dai.node.VideoEncoder)
    ve3 = pipeline.create(dai.node.VideoEncoder)

    ve1Out = pipeline.create(dai.node.XLinkOut)
    ve2Out = pipeline.create(dai.node.XLinkOut)
    ve3Out = pipeline.create(dai.node.XLinkOut)

    ve1Out.setStreamName(streamName[0])
    ve2Out.setStreamName(streamName[1])
    ve3Out.setStreamName(streamName[2])

    # Properties
    camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)
    monoLeft.setBoardSocket(dai.CameraBoardSocket.LEFT)
    monoRight.setBoardSocket(dai.CameraBoardSocket.RIGHT)
    # Create encoders, one for each camera, consuming the frames and encoding them using H.264 / H.265 encoding
    ve1.setDefaultProfilePreset(30, dai.VideoEncoderProperties.Profile.H264_HIGH)
    ve2.setDefaultProfilePreset(30, dai.VideoEncoderProperties.Profile.H264_HIGH)
    ve3.setDefaultProfilePreset(30, dai.VideoEncoderProperties.Profile.H264_HIGH)

    # Linking
    monoLeft.out.link(ve1.input)
    camRgb.video.link(ve2.input)
    monoRight.out.link(ve3.input)
    ve1.bitstream.link(ve1Out.input)
    ve2.bitstream.link(ve2Out.input)
    ve3.bitstream.link(ve3Out.input)
    return pipeline
@ray.remote
def recordCam(mxID,path,streamName,camName,time_limit):
    pipeline=getPipeLine(streamName)
    isDev1,deviceInfo=dai.Device.getDeviceByMxId(mxID)
    with dai.Device(pipeline,deviceInfo) as dev:

        # Output queues will be used to get the encoded data from the outputs defined above
        outQ1 = dev.getOutputQueue(name=streamName[0], maxSize=30, blocking=False)
        outQ2 = dev.getOutputQueue(name=streamName[1], maxSize=30, blocking=False)
        outQ3 = dev.getOutputQueue(name=streamName[2], maxSize=30, blocking=False)
        os.mkdir(path+'/'+camName)
        os.mkdir(path+'/'+camName+'/videos/')
        os.mkdir(path+'/'+camName+'/frames/')
        name=path+'/'+camName+'/videos/record'
        f_mono1=name+'_mono1.h264'
        f_mono2=name+'_mono2.h264'
        f_color=name+'_color.h264'
        start=time.time()
        print("RECORDING for"+ camName + " started at "+str(int(time.time())) + " seconds.")
        with open(f_mono1, 'wb') as fileMono1H264, open(f_color, 'wb') as fileColorH265, open(f_mono2, 'wb') as fileMono2H264:
#             print("Press Ctrl+C or interrupt kernel to stop encoding...")
            
            while (time.time()-start<time_limit+2):
#                 print(time.time()-start)
                try:
                    # Empty each queue
                    while outQ1.has():
#                         print("11"+str(outQ1.has()))
                        outQ1.get().getData().tofile(fileMono1H264)

                    while outQ2.has():
#                         print("21"+str(outQ2.has()))
                        outQ2.get().getData().tofile(fileColorH265)

                    while outQ3.has():
#                         print("31"+str(outQ3.has()))
                        outQ3.get().getData().tofile(fileMono2H264)
    
                except KeyboardInterrupt:
                    # Keyboard interrupt (Ctrl + C) detected
                    break
        print("Converting to mp4...")
        cmd = "ffmpeg -framerate 30 -i {} -c copy {}"

        os.system(cmd.format(f_mono1,name+'_mono1.mp4'))
        os.system(cmd.format(f_mono2, name+'_mono2.mp4'))
        os.system(cmd.format(f_color, name+"_color.mp4"))
        time.sleep(5)
        print("Conversion complete. Removing temporary files")

        os.remove(f_mono1)
        os.remove(f_mono2)
        os.remove(f_color)
        
        print("RECORDING for "+ camName + " ended at "+str(int(time.time())) + " seconds.")

In [57]:
streamName1=['ve1Out','ve2Out','ve3Out']
streamName2=['ve1Out1','ve2Out1','ve3Out1']
mxID1="14442C10913365D300"
mxID2="14442C10810665D300"
pipeline1=getPipeLine(streamName1)
pipeline2=getPipeLine(streamName2)
isDev1,dev1Info=dai.Device.getDeviceByMxId("14442C10913365D300")
isDev2,dev2Info=dai.Device.getDeviceByMxId("14442C10810665D300")
print(isDev1,isDev2)

True True


In [58]:
def recordCam1():
    print("RECORDING started at "+str(int(time.time())) + " seconds.")
    recordCam('14442C10913365D300',dev1Info,pipeline1,streamName1,'cam1',10)
def recordCam2():
    print("RECORDING started at "+str(int(time.time())) + " seconds.")
    recordCam('14442C10810665D300',dev2Info,pipeline2,streamName2,'cam2',10)

In [59]:
from multiprocessing import Process
from threading import Thread


In [60]:
# def run_records_in_parallel(funcs):
#     running_funcs=[Process(target=func) for func in funcs]
#     for running_func in running_funcs:
#         running_func.start()
#     for running_func in running_funcs:
#         running_func.join()
    

In [61]:
process1=Process(target=recordCam1)
process2=Process(target=recordCam2)
processes=[process1,process2]

In [28]:
for process in processes:
    process.start()
for process in processes:
    process.join()

PicklingError: Can't pickle <function recordCam1 at 0x000001988A7D9AF0>: it's not the same object as __main__.recordCam1

In [44]:
# from multiprocessing import Process

# def run_cpu_tasks_in_parallel(tasks):
#     running_tasks = [Process(target=task) for task in tasks]
#     for running_task in running_tasks:
#         running_task.start()
#     for running_task in running_tasks:
#         running_task.join()

# run_cpu_tasks_in_parallel([
#     recordCam1,
#     recordCam2
# ])

In [45]:
import ray

In [62]:

ray.init()
month,day,year=time.localtime().tm_mon,time.localtime().tm_mday,time.localtime().tm_year
hour,minutes,seconds=time.localtime().tm_hour,time.localtime().tm_min,time.localtime().tm_sec
dir_name=str(month)+'_'+str(day)+'_'+str(year)+'_'+str(hour)+str(minutes)
path=FDIR+'/'+dir_name
os.mkdir(path)
ray.get([recordCam.remote('14442C10913365D300',path,streamName1,'cam1',10),
         recordCam.remote('14442C10810665D300',path,streamName2,'cam2',10)])
ray.shutdown()

 pid=7756) RECORDING forcam2 started at 1656984924 seconds.
 pid=9752) RECORDING forcam1 started at 1656984924 seconds.


(recordCam
 pid=7756) ffmpeg version 2022-02-07-git-04cc7a5548-full_build-www.gyan.dev Copyright (c) 2000-2022 the FFmpeg developers
(recordCam
 pid=7756)   built with gcc 11.2.0 (Rev7, Built by MSYS2 project)
(recordCam
 pid=7756)   configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --e

(recordCam
 pid=7756) Converting to mp4...


 pid=7756) ffmpeg version 2022-02-07-git-04cc7a5548-full_build-www.gyan.dev Copyright (c) 2000-2022 the FFmpeg developers
 pid=7756)   built with gcc 11.2.0 (Rev7, Built by MSYS2 project)
 pid=7756)   configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enab

(recordCam
 pid=9752) Converting to mp4...


(recordCam
 pid=9752) ffmpeg version 2022-02-07-git-04cc7a5548-full_build-www.gyan.dev Copyright (c) 2000-2022 the FFmpeg developers
(recordCam
 pid=9752)   built with gcc 11.2.0 (Rev7, Built by MSYS2 project)
(recordCam
 pid=9752)   configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --e

 pid=7756) Conversion complete. Removing temporary files
 pid=7756) RECORDING for cam2 ended at 1656984941 seconds.
 pid=9752) Conversion complete. Removing temporary files
 pid=9752) RECORDING for cam1 ended at 1656984941 seconds.


[None, None]

In [64]:
%pip list

Package                            Version                                                Editable project location
---------------------------------- ------------------------------------------------------ ------------------------------------------
absl-py                            1.0.0
alabaster                          0.7.12
anaconda-client                    1.9.0
anaconda-navigator                 2.1.1
anaconda-project                   0.10.1
anyio                              2.2.0
appdirs                            1.4.4
argcomplete                        1.12.1
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              0.13.1
asn1crypto                         1.4.0
astroid                            2.6.6
astropy                            4.3.1
astunparse                         1.6.3
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              21.2.0
audiore

You should consider upgrading via the 'C:\Users\gehan\anaconda3\python.exe -m pip install --upgrade pip' command.


In [65]:
%conda list

# packages in environment at C:\Users\gehan\anaconda3:
#
# Name                    Version                   Build  Channel
_ipyw_jlab_nb_ext_conf    0.1.0            py39haa95532_0  
absl-py                   1.0.0                    pypi_0    pypi
alabaster                 0.7.12             pyhd3eb1b0_0  
anaconda                  2021.11                  py39_0  
anaconda-client           1.9.0            py39haa95532_0  
anaconda-navigator        2.1.1                    py39_0  
anaconda-project          0.10.1             pyhd3eb1b0_0  
anyio                     2.2.0            py39haa95532_2  
appdirs                   1.4.4              pyhd3eb1b0_0  
argh                      0.26.2           py39haa95532_0  
argon2-cffi               20.1.0           py39h2bbff1b_1  
arrow                     0.13.1           py39haa95532_0  
asn1crypto                1.4.0                      py_0  
astroid                   2.6.6            py39haa95532_0  
astropy                   4.3.


qdarkstyle                3.0.2              pyhd3eb1b0_0  
qstylizer                 0.1.10             pyhd3eb1b0_0  
qt                        5.9.7            vc14h73c81de_0  
qtawesome                 1.0.2              pyhd3eb1b0_0  
qtconsole                 5.1.1              pyhd3eb1b0_0  
qtpy                      1.10.0             pyhd3eb1b0_0  
ray                       1.10.0                   pypi_0    pypi
redis                     4.1.4                    pypi_0    pypi
regex                     2021.8.3         py39h2bbff1b_0  
requests                  2.26.0             pyhd3eb1b0_0  
requests-oauthlib         1.3.0                    pypi_0    pypi
resampy                   0.2.2                    pypi_0    pypi
rfc3987                   1.3.8                    pypi_0    pypi
rope                      0.19.0             pyhd3eb1b0_0  
rtree                     0.9.7            py39h2eaa2aa_1  
ruamel-yaml               0.17.17                  pypi_0    pypi
rua